In [ ]:
# importing and installing 
import pandas as pd
import openai
import pandas as pd
import time




In [ ]:
# loading dataset



# Directory path where the thread files are stored
#path = "../newdatasets/datasetsmine/updated_data.csv"
#data = pd.read_csv(path)


# Directory path where the thread files are stored
path = "../newdatasets/datasetgroup/data_with_summarizes2.csv"
data = pd.read_csv(path)

# Check the dataset
print(f"Dataset loaded successfully with {len(data)} rows.")
print(data.head())


In [ ]:
# Select the first 2000 rows but keep all columns
sample_data = data.head(5)

# Check the sample
sample_data[['cleaned_text','cleaned_body']].head(5)

In [ ]:
import openai

In [ ]:
'''
import time

# Set up OpenAI API key
openai.api_key = "my_key"

# Function to get GPT label for a single post and comment
def get_gpt_label(post, comment):
    try:
        # Prepare the prompt
        prompt = f"""
        You are an AI tasked with classifying the conceptual agreement and relationship between the following post and comment between about climate change. The possible labels are:
        - 1: Agree (the comment conceptually agrees with, and supports the post's argument and opinion)
        - 0: Neutral/Unrelated (the comment neither agrees nor disagrees conceptually but shifts focus or introduces unrelated ideas)
        - -1: Disagree (the comment rejects or challenges the post's argument).

        Example 1:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "I agree, but we should also focus on methane reductions."
        Label: 1 (Agree)

        Example 2:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "Climate change isn't real. This is all a hoax."
        Label: -1 (Disagree)

        Example 3:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "I think renewable energy is great, but it's expensive."
        Label: 0 (Neutral/Unrelated)

        Classify the following post-comment pair:
        Post: "{post}"

        Comment: "{comment}"

        Provide the label (1, 0, or -1) in one column and a brief explanation for your classification in another column.
        """

        # Call OpenAI GPT API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Extract the response content
        label = response['choices'][0]['message']['content'].strip()
        return int(label)  # Convert to integer

    except Exception as e:
        print(f"Error: {e}")
        return None  # Return None if there's an error




# Shuffle the dataset without resetting the index
shuffled_data = data.sample(frac=1, random_state=42)
shuffled_data['original_index'] = shuffled_data.index  # Add the original index as a new column


# Process the dataset (use a sample for testing, e.g., first 2000 rows)
sample_data = shuffled_data.head(3000).copy()

# Add a new column for GPT labels
sample_data['label_agreement'] = None
sample_data['explanation'] = None


# Iterate through each post and comment pair
for i, row in sample_data.iterrows():
    post = row['cleaned_text']
    comment = row['cleaned_body']
    
    # Get GPT label
    label = get_gpt_label(post, comment)
    sample_data.at[i, 'label'] = label
    sample_data.at[i, 'explanation'] = explanation
    
    
    # Optional: Print progress
    if i % 10 == 0:
        print(f"Processed {i} rows...")

    # Avoid hitting API rate limits (adjust delay as needed)
    time.sleep(1.5)

# Save the labeled dataset
output_path = "labeled_sample_data.csv"
sample_data.to_csv(output_path, index=False)
print(f"Labeled dataset saved to {output_path}")
'''

In [3]:
# for new dataset thissssssssssss


import pandas as pd
import time
import openai

# Set up OpenAI API key
openai.api_key = "my_key"

# Load dataset
data_path = "../newdatasets/datasetgroup/data_with_summarizes2.csv"
data = pd.read_csv(data_path)

# Ensure columns are strings
data['cleaned_text'] = data['cleaned_text'].fillna("").astype(str)
data['cleaned_body'] = data['cleaned_body'].fillna("").astype(str)

# Shuffle the dataset without resetting the index
shuffled_data = data.sample(frac=1, random_state=42)
shuffled_data['original_index'] = shuffled_data.index  # Preserve the original index

# Select a sample of 5 rows for labeling
sample_data = shuffled_data.head(5).copy()

# Add columns for GPT results
sample_data['label_agreement'] = None
sample_data['explanation'] = None


# Function to get GPT label for a single post and comment
def get_gpt_label(post, comment):
    try:
        # Prepare the prompt
        prompt = f"""
        You are an AI tasked with classifying the conceptual agreement and relationship between the following post and comment about climate change. The possible labels are:
        - 1: Agree (the comment conceptually agrees with, and supports the post's argument and opinion)
        - 0: Neutral/Unrelated (the comment neither agrees nor disagrees conceptually but shifts focus or introduces unrelated ideas)
        - -1: Disagree (the comment rejects or challenges the post's argument).

        Classify the following post-comment pair:
        Post: "{post}"

        Comment: "{comment}"

        Provide the label (1, 0, or -1) and a brief explanation for your classification, separated by a colon.
        Example: "1: The comment agrees with the post's argument about reducing CO2 emissions."
        """
        
        # Call OpenAI GPT API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0
        )
        
        # Extract and parse the response
        result = response['choices'][0]['message']['content'].strip()
        label, explanation = result.split(":", 1)
        return int(label.strip()), explanation.strip()
    
    except Exception as e:
        print(f"Error processing post-comment pair: {e}")
        return None, "Error encountered during processing."


# Iterate through each post and comment pair
for i, row in sample_data.iterrows():
    post = row['cleaned_text']
    comment = row['cleaned_body']
    
    # Get GPT label and explanation
    label, explanation = get_gpt_label(post, comment)
    sample_data.at[i, 'label_agreement'] = label
    sample_data.at[i, 'explanation'] = explanation
    
    # Print progress for debugging
    print(f"\nPost:\n{post[:300]}...")
    print(f"Comment:\n{comment[:300]}...")
    print(f"Label: {label}")
    print(f"Explanation: {explanation}\n")
    
    # Optional: Print progress periodically
    if i % 10 == 0:
        print(f"Processed {i + 1}/{len(sample_data)} rows...")

    # Avoid hitting API rate limits (adjust delay as needed)
    time.sleep(2)


# Save the labeled sample dataset
labeled_sample_path = "../newdatasets/datasetgroup/output/labeled_sample_data.csv"
sample_data.to_csv(labeled_sample_path, index=False)
print(f"Labeled sample dataset saved to {labeled_sample_path}")

# Save the shuffled_data
shuffled_data_path = "../newdatasets/datasetgroup/output/shuffled_data.csv"
shuffled_data.to_csv(shuffled_data_path, index=False)  # Use the DataFrame, not a string
print(f"Shuffled dataset saved to {shuffled_data_path}")

# Save the remaining unlabeled data
unlabeled_data = shuffled_data.iloc[3000:].copy()  # Ensure the remaining rows are selected correctly
unlabeled_path = "../newdatasets/datasetgroup/output/unlabeled_data.csv"
unlabeled_data.to_csv(unlabeled_path, index=False)
print(f"Unlabeled data saved to {unlabeled_path}")



Post:
when you ask the average conservative why they dont seem to take climate change seriously, you often hear something along the lines of well, theyve all told us that the world would end in 10 years, and it didnt, and since that didnt happen, my trust is now completely broken and i feel entitled not t...
Comment:
if you already believe yourself to be intelligent, which most people do-- how much more intelligent do you expect other people to be when dealing with models that have hundreds or thousands of complex variables? is that something you feel like you could do extremely accurately? is that something you...
Label: -1
Explanation: The comment disagrees with the post's argument. The post argues that conservatives' disbelief in climate change is due to misinformation and lack of credible sources, while the comment suggests that the disbelief is due to the complexity of climate models and a refusal to accept new evidence or correct previous misconceptions.


Post:
i think a lot of

In [5]:
import pandas as pd
import time
import openai

# Set up OpenAI API key
openai.api_key = "my_key"

# Load dataset
data_path = "../newdatasets/datasetgroup/data_with_summarizes2.csv"
data = pd.read_csv(data_path)

# Ensure columns are strings
data['cleaned_text'] = data['cleaned_text'].fillna("").astype(str)
data['cleaned_body'] = data['cleaned_body'].fillna("").astype(str)

# Shuffle the dataset without resetting the index
shuffled_data = data.sample(frac=1, random_state=42)
shuffled_data['original_index'] = shuffled_data.index  # Preserve the original index

# Select a sample of 3000 rows for labeling
sample_data = shuffled_data.head(3000).copy()

# Add columns for GPT results
sample_data['label_agreement'] = None
sample_data['explanation'] = None


# Function to get GPT label for a single post and comment
def get_gpt_label(post, comment):
    try:
        # Prepare the prompt
        prompt = f"""
        You are an AI tasked with classifying the conceptual agreement and relationship between the following post and comment about climate change. The possible labels are:
        - 1: Agree (the comment conceptually agrees with, and supports the post's argument and opinion)
        - 0: Neutral/Unrelated (the comment neither agrees nor disagrees conceptually but shifts focus or introduces unrelated ideas)
        - -1: Disagree (the comment rejects or challenges the post's argument).

        Classify the following post-comment pair:
        Post: "{post}"

        Comment: "{comment}"

        Provide the label (1, 0, or -1) and a brief explanation for your classification, separated by a colon.
        Example: "1: The comment agrees with the post's argument about reducing CO2 emissions."
        """
        
        # Call OpenAI GPT API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150,
            temperature=0
        )
        
        # Extract and parse the response
        result = response['choices'][0]['message']['content'].strip()
        
        # Split the response into label and explanation
        try:
            label, explanation = result.split(":", 1)
            label = int(label.strip().strip('"'))  # Handle potential formatting quirks
            explanation = explanation.strip().strip('"')  # Clean explanation
        except ValueError:
            # If the response isn't formatted as expected, log and return defaults
            print(f"Unexpected response format: {result}")
            label, explanation = None, f"Error parsing response: {result}"
        
        return label, explanation
    
    except Exception as e:
        print(f"Error processing post-comment pair: {e}")
        return None, f"Error encountered during processing: {e}"


# Iterate through each post and comment pair
for i, row in sample_data.iterrows():
    post = row['cleaned_text']
    comment = row['cleaned_body']
    
    # Get GPT label and explanation
    label, explanation = get_gpt_label(post, comment)
    sample_data.at[i, 'label_agreement'] = label
    sample_data.at[i, 'explanation'] = explanation
    
    # Print progress for debugging
    print(f"\nPost:\n{post[:300]}...")
    print(f"Comment:\n{comment[:300]}...")
    print(f"Label: {label}")
    print(f"Explanation: {explanation}\n")
    
    # Optional: Print progress periodically
    if i % 10 == 0:
        print(f"Processed {i + 1}/{len(sample_data)} rows...")

    # Avoid hitting API rate limits (adjust delay as needed)
    time.sleep(2)

# Save the labeled sample dataset
labeled_sample_path = "../newdatasets/datasetgroup/output/labeled_sample_data.csv"
sample_data.to_csv(labeled_sample_path, index=False)
print(f"Labeled sample dataset saved to {labeled_sample_path}")

# Save the shuffled_data
shuffled_data_path = "../newdatasets/datasetgroup/output/shuffled_data.csv"
shuffled_data.to_csv(shuffled_data_path, index=False)  # Use the DataFrame, not a string
print(f"Shuffled dataset saved to {shuffled_data_path}")

# Save the remaining unlabeled data
unlabeled_data = shuffled_data.iloc[3000:].copy()  # Ensure the remaining rows are selected correctly
unlabeled_path = "../newdatasets/datasetgroup/output/unlabeled_data.csv"
unlabeled_data.to_csv(unlabeled_path, index=False)
print(f"Unlabeled data saved to {unlabeled_path}")



Post:
when you ask the average conservative why they dont seem to take climate change seriously, you often hear something along the lines of well, theyve all told us that the world would end in 10 years, and it didnt, and since that didnt happen, my trust is now completely broken and i feel entitled not t...
Comment:
if you already believe yourself to be intelligent, which most people do-- how much more intelligent do you expect other people to be when dealing with models that have hundreds or thousands of complex variables? is that something you feel like you could do extremely accurately? is that something you...
Label: 1
Explanation: The comment agrees with the post's argument that the misunderstanding and misinterpretation of scientific data and predictions about climate change have led to a lack of trust and serious consideration of the issue. The comment further elaborates on this by suggesting that people may not fully understand the complexity of climate models and may reject 

In [ ]:
# for binary:

'''
import time

# Set up OpenAI API key
openai.api_key = "my_key"


# Function to get GPT label for a single post and comment
def get_gpt_label(post, comment):
    try:
        # Prepare the prompt
        prompt = f"""
        Analyze the semantic relationship between the following post and comment.
        Post: {post}
        Comment: {comment}
        
        Does the comment:
        +1: Agree with the post,
        0: Disagree with the post,
        
        
        Respond with one number only: +1 or 0.
        """
        
        # Call OpenAI GPT API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Extract the response content
        label = response['choices'][0]['message']['content'].strip()
        return int(label)  # Convert to integer

    except Exception as e:
        print(f"Error: {e}")
        return None  # Return None if there's an error

# Process the dataset (use a sample for testing, e.g., first 2000 rows)
sample_data = data.head(2000).copy()

# Add a new column for GPT labels
sample_data['label_binary'] = None

# Iterate through each post and comment pair
for i, row in sample_data.iterrows():
    post = row['cleaned_text']
    comment = row['cleaned_body']
    
    # Get GPT label
    label = get_gpt_label(post, comment)
    sample_data.at[i, 'label_binary'] = label
    
    # Optional: Print progress
    if i % 10 == 0:
        print(f"Processed {i} rows...")

    # Avoid hitting API rate limits (adjust delay as needed)
    time.sleep(1.5)

# Save the labeled dataset
output_path = "labeled_sample_data2.csv"
sample_data.to_csv(output_path, index=False)
print(f"Labeled dataset saved to {output_path}")
'''
